In [1]:
%run _standard_imports.ipynb
%run _plotting_setup.ipynb

python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.2
scipy 0.18.1
pandas 0.19.0
numexpr 2.6.1
pysam 0.8.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.2.0
h5py 2.6.0
tables 3.3.0


In [2]:
output_dir = '/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161204_variants_counts_summaries'
!mkdir -p {output_dir}

normalised_annotated_vcf_format = collections.OrderedDict()
normalised_annotated_vcf_format['Pf60'] = '/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/\
20161125_Pf60_final_vcfs/intermediate_files/SNP_INDEL_%s.combined.filtered.annotated.normalised.vcf.gz'
normalised_annotated_vcf_format['Pv30'] = '/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pv3.0/\
20161130_Pv30_final_vcfs/intermediate_files/SNP_INDEL_%s.combined.filtered.annotated.normalised.vcf.gz'

normalised_annotated_vcf_fofn = collections.OrderedDict()
normalised_annotated_wg_vcf_fn = collections.OrderedDict()
for release in normalised_annotated_vcf_format:
    normalised_annotated_vcf_fofn[release] = '%s/%s_normalised_annotated_vcf.fofn' % (output_dir, release)
    normalised_annotated_wg_vcf_fn[release] = "%s/%s_SNP_INDEL_WG.combined.filtered.annotated.normalised.vcf.gz" % (output_dir, release)

GENOME_FN = collections.OrderedDict()
genome_fn = collections.OrderedDict()
genome = collections.OrderedDict()
GENOME_FN['Pf60'] = "/lustre/scratch116/malaria/pfalciparum/resources/Pfalciparum.genome.fasta"
GENOME_FN['Pv30'] = "/lustre/scratch109/malaria/pvivax/resources/gatk/PvivaxP01.genome.fasta"
genome_fn['Pf60'] = "%s/Pfalciparum.genome.fasta" % output_dir
genome_fn['Pv30'] = "%s/PvivaxP01.genome.fasta" % output_dir


# sites_annotation_pf60_fn = '/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161125_Pf60_final_vcfs/vcf/SNP_INDEL_WG.combined.filtered.annotation.vcf.gz'
hdf_fn = collections.OrderedDict()
hdf_fn['Pf60'] = '/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161128_HDF5_build/hdf5/Pf_60_npy_PID_a12.h5'
hdf_fn['Pv30'] = '/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161201_Pv_30_HDF5_build/hdf5/Pv_30.h5'


In [9]:
normalised_annotated_vcf_fofn['Pf60']

'/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161204_variants_counts_summaries/Pf60_normalised_annotated_vcf.fofn'

In [12]:
for release in GENOME_FN:
    !cp {GENOME_FN[release]} {genome_fn[release]}
    genome[release] = pyfasta.Fasta(genome_fn[release])

In [13]:
genome

OrderedDict([('Pf60', <pyfasta.fasta.Fasta at 0x2ad403f81518>),
             ('Pv30', <pyfasta.fasta.Fasta at 0x2ad403f815f8>)])

In [14]:
!ls -l {normalised_annotated_vcf_format['Pf60'] % 'Pf3D7_01_v3'}

-rw-rw-r-- 1 rp7 team112 7077195 Nov 25 16:37 /lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161125_Pf60_final_vcfs/intermediate_files/SNP_INDEL_Pf3D7_01_v3.combined.filtered.annotated.normalised.vcf.gz


In [15]:
!ls -l {normalised_annotated_vcf_format['Pv30'] % 'PvP01_01_v1'}

-rw-rw-r-- 1 rp7 team112 2907232 Nov 30 21:46 /lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pv3.0/20161130_Pv30_final_vcfs/intermediate_files/SNP_INDEL_PvP01_01_v1.combined.filtered.annotated.normalised.vcf.gz


In [4]:
!bjobs

JOBID   USER    STAT  QUEUE      FROM_HOST   EXEC_HOST   JOB_NAME   SUBMIT_TIME
9482292 rp7     RUN   basement   malsrv2     16*bc-31-1- bash       Dec  5 09:34
9461872 rp7     PEND  basement   malsrv2                 bash       Dec  4 10:53
9462674 rp7     PEND  basement   malsrv2                 bash       Dec  4 11:48
9482075 rp7     PEND  basement   malsrv2                 bash       Dec  5 09:31
9485145 rp7     PEND  basement   malsrv2                 bash       Dec  5 10:03


In [14]:
for release in normalised_annotated_vcf_fofn:
    fo = open(normalised_annotated_vcf_fofn[release], 'w')
    for chrom in sorted(list(genome[release].keys())):
        vcf_fn = normalised_annotated_vcf_format[release] % chrom
        print(vcf_fn, file=fo)
    fo.close()

In [3]:
hdf = collections.OrderedDict()
for release in hdf_fn:
    hdf[release] = h5py.File(hdf_fn[release], 'r')


In [31]:
def num_unique(values):
    return(len(np.unique(values)))

def determine_variant_counts(variants=variants['Pf60'], VQSLOD_threshold=0.0, groupby_variables = ['VARIANT_TYPE', 'CDS'], ascending=None,
                             number_of_samples=number_of_samples['Pf60']):
    if ascending is None:
        ascending = [True if x == 'MULTIALLELIC' else False for x in groupby_variables]
        
    df_variants = pd.DataFrame(
        {
            'CHROM': pd.Series(variants['CHROM'], dtype="category"),
            'POS': pd.Series(variants['POS']),
            'AC': pd.Series(variants['AC']),
            'AN': pd.Series(variants['AN']),
            'NRAF': pd.Series(variants['AC'] / variants['AN']),
            'Missingness': pd.Series(((number_of_samples * 2) - variants['AN']) / (number_of_samples * 2)),
            'CDS': pd.Series(variants['CDS'], dtype="category"),
            'VARIANT_TYPE': pd.Series(variants['VARIANT_TYPE'], dtype="category"),
            'MULTIALLELIC': pd.Series(variants['MULTIALLELIC'], dtype="category"),
        }
    )
#     For the following, Pf60 has FILTER_SubtelomericRepeat but Pv30 doesn't
    if 'FILTER_SubtelomericRepeat' in variants.dtype.names:
        ix_good_calls = (
            (variants['VARIANT_TYPE'] != b'') &
            (~variants['FILTER_Centromere']) &
            (~variants['FILTER_InternalHypervariable']) &
            (~variants['FILTER_SubtelomericHypervariable']) &
            (~variants['FILTER_SubtelomericRepeat']) &
            (variants['VQSLOD'] > VQSLOD_threshold)
        )
    else:
        ix_good_calls = (
            (variants['VARIANT_TYPE'] != b'') &
            (~variants['FILTER_Centromere']) &
            (~variants['FILTER_InternalHypervariable']) &
            (~variants['FILTER_SubtelomericHypervariable']) &
            (variants['VQSLOD'] > VQSLOD_threshold)
        )

    df_variants['PASS'] = ix_good_calls
    
    for i, column in enumerate(['CHROM', 'VARIANT_TYPE', 'MULTIALLELIC']):
        df_variants[column].cat.categories = [
            x.decode('ascii') for x in df_variants[column].cat.categories
        ]
    df_variants['CHROM_POS'] = df_variants['CHROM'].str.cat(df_variants['POS'].astype(str), sep='_')

    if len(groupby_variables) == 0:
#         temp = pd.DataFrame(variant_type_sum)
        df_variants['Total'] = 'Total'
        groupby_variables = ['Total']
    
#         temp.set_index('Total', append=True, inplace=True)
#         variant_type_sum = temp.ix[:, 0]
    
    gb_all = df_variants.groupby(groupby_variables)
    gb_pass = df_variants[df_variants['PASS']].groupby(groupby_variables)
    
    df_summary = pd.concat(
        [
            gb_all['CHROM_POS'].agg(num_unique).rename("Discovered variant positions"),
            gb_pass['CHROM_POS'].agg(num_unique).rename("Pass variant positions"),
            gb_pass.size().rename("Alleles"),
            gb_pass['NRAF'].mean().rename("Mean NRAF"),
            gb_pass['Missingness'].mean().rename("Missingness"),
        ],
        axis=1
    )
    df_summary["% pass"] = df_summary["Pass variant positions"] / df_summary["Discovered variant positions"]
    df_summary["Alleles per position"] = df_summary["Alleles"] / df_summary["Pass variant positions"]
    df_summary["Variants per sample"] = df_summary["Alleles"] * df_summary["Mean NRAF"]
    
    df_summary.sortlevel(range(len(groupby_variables)), ascending=ascending, inplace=True)
    df_summary.reset_index(inplace=True)
        
    if 'MULTIALLELIC' in df_summary.columns:
        df_summary = df_summary[df_summary['MULTIALLELIC'] != 'MIS']
    
    return(df_summary)
    


In [ ]:
# annotation_fn = "%s/vcf/SNP_INDEL_WG.sites.annotation.txt.gz" % (output_dir)

# !bcftools norm -m -any --fasta-ref {GENOME_FN} {sites_only_vcf_fn}| \
# bcftools view --include 'ALT!="*"' | \
# bcftools annotate -h {annotations_header_fn} \
# -a {annotation_fn} -c CHROM,POS,REF,ALT,CDS,VARIANT_TYPE,MULTIALLELIC \
# --include 'INFO/AC>0' \
# --remove ^INFO/AC,INFO/AN,INFO/AF,INFO/VQSLOD,^FORMAT/GT,FORMAT/AD,FORMAT/GQ -Oz -o {sites_annotation_vcf_fn}

# !bcftools index --tbi {sites_annotation_vcf_fn}

In [15]:
for release in normalised_annotated_vcf_fofn:
    print(release)
    !bcftools concat --file-list {normalised_annotated_vcf_fofn[release]} -Oz -o {normalised_annotated_wg_vcf_fn[release]}
    !bcftools index --tbi {normalised_annotated_wg_vcf_fn[release]}

Pf60
Pv30


In [16]:
def create_variants_npy(vcf_fn = normalised_annotated_wg_vcf_fn['Pf60']):
    output_dir = '%s.vcfnp_cache' % vcf_fn
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    vcfnp.variants(
        vcf_fn,
        fields=['CHROM', 'POS', 'REF', 'ALT', 'CDS', 'VARIANT_TYPE', 'MULTIALLELIC',
                'AC', 'AN', 'FILTER', 'VQSLOD'],
        dtypes={
            'REF':                      'a600',
            'ALT':                      'a600',
            'VARIANT_TYPE':             'a3',
            'CDS':                      bool,
            'MULTIALLELIC':             'a2',
        },
        arities={
            'ALT':   1,
            'AC':    1,
        },
        flatten_filter=True,
        progress=100000,
        verbose=True,
        cache=True,
        cachedir=output_dir
    )


In [18]:
for release in normalised_annotated_wg_vcf_fn:
    print(release)
    create_variants_npy(normalised_annotated_wg_vcf_fn[release])

[vcfnp] 2016-12-05 10:10:22.121869 :: caching is enabled
[vcfnp] 2016-12-05 10:10:22.124020 :: no cache file found
[vcfnp] 2016-12-05 10:10:22.125021 :: building array


Pf60


[vcfnp] 2016-12-05 10:10:26.304265 :: 100000 rows in 4.16s; batch in 4.16s (24032 rows/s)
[vcfnp] 2016-12-05 10:10:30.474542 :: 200000 rows in 8.33s; batch in 4.17s (23979 rows/s)
[vcfnp] 2016-12-05 10:10:34.585391 :: 300000 rows in 12.44s; batch in 4.11s (24325 rows/s)
[vcfnp] 2016-12-05 10:10:38.760979 :: 400000 rows in 16.62s; batch in 4.18s (23948 rows/s)
[vcfnp] 2016-12-05 10:10:42.892628 :: 500000 rows in 20.75s; batch in 4.13s (24203 rows/s)
[vcfnp] 2016-12-05 10:10:47.057964 :: 600000 rows in 24.91s; batch in 4.17s (24007 rows/s)
[vcfnp] 2016-12-05 10:10:51.176441 :: 700000 rows in 29.03s; batch in 4.12s (24280 rows/s)
[vcfnp] 2016-12-05 10:10:55.317137 :: 800000 rows in 33.17s; batch in 4.14s (24150 rows/s)
[vcfnp] 2016-12-05 10:10:59.483026 :: 900000 rows in 37.34s; batch in 4.17s (24004 rows/s)
[vcfnp] 2016-12-05 10:11:03.635017 :: 1000000 rows in 41.49s; batch in 4.15s (24084 rows/s)
[vcfnp] 2016-12-05 10:11:07.773450 :: 1100000 rows in 45.63s; batch in 4.14s (24163 rows/s)

Pv30


[vcfnp] 2016-12-05 10:17:41.735353 :: 100000 rows in 4.12s; batch in 4.12s (24278 rows/s)
[vcfnp] 2016-12-05 10:17:45.870720 :: 200000 rows in 8.25s; batch in 4.14s (24181 rows/s)
[vcfnp] 2016-12-05 10:17:50.016051 :: 300000 rows in 12.40s; batch in 4.15s (24123 rows/s)
[vcfnp] 2016-12-05 10:17:54.145158 :: 400000 rows in 16.53s; batch in 4.13s (24218 rows/s)
[vcfnp] 2016-12-05 10:17:58.309528 :: 500000 rows in 20.69s; batch in 4.16s (24013 rows/s)
[vcfnp] 2016-12-05 10:18:02.417453 :: 600000 rows in 24.80s; batch in 4.11s (24343 rows/s)
[vcfnp] 2016-12-05 10:18:06.572166 :: 700000 rows in 28.96s; batch in 4.15s (24069 rows/s)
[vcfnp] 2016-12-05 10:18:10.680115 :: 800000 rows in 33.06s; batch in 4.11s (24343 rows/s)
[vcfnp] 2016-12-05 10:18:14.798947 :: 900000 rows in 37.18s; batch in 4.12s (24278 rows/s)
[vcfnp] 2016-12-05 10:18:18.904510 :: 1000000 rows in 41.29s; batch in 4.11s (24357 rows/s)
[vcfnp] 2016-12-05 10:18:23.031385 :: 1100000 rows in 45.41s; batch in 4.13s (24231 rows/s)

In [19]:
variants = collections.OrderedDict()
for release in normalised_annotated_wg_vcf_fn:
    variants[release] = np.load("%s.vcfnp_cache/variants.npy" % normalised_annotated_wg_vcf_fn[release])

In [20]:
number_of_samples = collections.OrderedDict()
number_of_samples['Pf60'] = 7182
number_of_samples['Pv30'] = 1001


In [31]:
def num_unique(values):
    return(len(np.unique(values)))

def determine_variant_counts(variants=variants['Pf60'], VQSLOD_threshold=0.0, groupby_variables = ['VARIANT_TYPE', 'CDS'], ascending=None,
                             number_of_samples=number_of_samples['Pf60']):
    if ascending is None:
        ascending = [True if x == 'MULTIALLELIC' else False for x in groupby_variables]
        
    df_variants = pd.DataFrame(
        {
            'CHROM': pd.Series(variants['CHROM'], dtype="category"),
            'POS': pd.Series(variants['POS']),
            'AC': pd.Series(variants['AC']),
            'AN': pd.Series(variants['AN']),
            'NRAF': pd.Series(variants['AC'] / variants['AN']),
            'Missingness': pd.Series(((number_of_samples * 2) - variants['AN']) / (number_of_samples * 2)),
            'CDS': pd.Series(variants['CDS'], dtype="category"),
            'VARIANT_TYPE': pd.Series(variants['VARIANT_TYPE'], dtype="category"),
            'MULTIALLELIC': pd.Series(variants['MULTIALLELIC'], dtype="category"),
        }
    )
#     For the following, Pf60 has FILTER_SubtelomericRepeat but Pv30 doesn't
    if 'FILTER_SubtelomericRepeat' in variants.dtype.names:
        ix_good_calls = (
            (variants['VARIANT_TYPE'] != b'') &
            (~variants['FILTER_Centromere']) &
            (~variants['FILTER_InternalHypervariable']) &
            (~variants['FILTER_SubtelomericHypervariable']) &
            (~variants['FILTER_SubtelomericRepeat']) &
            (variants['VQSLOD'] > VQSLOD_threshold)
        )
    else:
        ix_good_calls = (
            (variants['VARIANT_TYPE'] != b'') &
            (~variants['FILTER_Centromere']) &
            (~variants['FILTER_InternalHypervariable']) &
            (~variants['FILTER_SubtelomericHypervariable']) &
            (variants['VQSLOD'] > VQSLOD_threshold)
        )

    df_variants['PASS'] = ix_good_calls
    
    for i, column in enumerate(['CHROM', 'VARIANT_TYPE', 'MULTIALLELIC']):
        df_variants[column].cat.categories = [
            x.decode('ascii') for x in df_variants[column].cat.categories
        ]
    df_variants['CHROM_POS'] = df_variants['CHROM'].str.cat(df_variants['POS'].astype(str), sep='_')

    if len(groupby_variables) == 0:
#         temp = pd.DataFrame(variant_type_sum)
        df_variants['Total'] = 'Total'
        groupby_variables = ['Total']
    
#         temp.set_index('Total', append=True, inplace=True)
#         variant_type_sum = temp.ix[:, 0]
    
    gb_all = df_variants.groupby(groupby_variables)
    gb_pass = df_variants[df_variants['PASS']].groupby(groupby_variables)
    
    df_summary = pd.concat(
        [
            gb_all['CHROM_POS'].agg(num_unique).rename("Discovered variant positions"),
            gb_pass['CHROM_POS'].agg(num_unique).rename("Pass variant positions"),
            gb_pass.size().rename("Alleles"),
            gb_pass['NRAF'].mean().rename("Mean NRAF"),
            gb_pass['Missingness'].mean().rename("Missingness"),
        ],
        axis=1
    )
    df_summary["% pass"] = df_summary["Pass variant positions"] / df_summary["Discovered variant positions"]
    df_summary["Alleles per position"] = df_summary["Alleles"] / df_summary["Pass variant positions"]
    df_summary["Variants per sample"] = df_summary["Alleles"] * df_summary["Mean NRAF"]
    
    df_summary.sortlevel(range(len(groupby_variables)), ascending=ascending, inplace=True)
    df_summary.reset_index(inplace=True)
        
    if 'MULTIALLELIC' in df_summary.columns:
        df_summary = df_summary[df_summary['MULTIALLELIC'] != 'MIS']
    
    return(df_summary)
    


In [56]:
len(hdf['Pf60']['samples'])

7182

In [60]:
np.array(list(hdf['Pf60']['variants'].keys()))

array(['AC', 'AF', 'ALT', 'AN', 'CDS', 'CHROM', 'DP', 'FILTER_Centromere',
       'FILTER_InternalHypervariable', 'FILTER_Low_VQSLOD', 'FILTER_PASS',
       'FILTER_SubtelomericHypervariable', 'FILTER_SubtelomericRepeat',
       'FS', 'MQ', 'MULTIALLELIC', 'POS', 'QD', 'QUAL', 'REF',
       'RegionType', 'SNPEFF_AMINO_ACID_CHANGE', 'SNPEFF_CODON_CHANGE',
       'SNPEFF_EFFECT', 'SNPEFF_EXON_ID', 'SNPEFF_FUNCTIONAL_CLASS',
       'SNPEFF_GENE_NAME', 'SNPEFF_IMPACT', 'SNPEFF_TRANSCRIPT_ID', 'SOR',
       'VARIANT_TYPE', 'VQSLOD', 'VariantType', 'is_snp', 'num_alleles',
       'svlen'], 
      dtype='<U32')

In [71]:
print(hdf['Pf60']['variants']['AC'][0])
print(hdf['Pf60']['variants']['AC'][1])
print(hdf['Pf60']['variants']['AC'][2])
print(hdf['Pf60']['variants']['AC'][7])

[2 0 0 0 0 0]
[ 3 11  0  0  0  0]
[21 14  0  0  0  0]
[27  1  9  0  0  0]


In [73]:
def num_AC(values):
    return(np.count_nonzero(values > 0))

print(num_AC(hdf['Pf60']['variants']['AC'][0]))
print(num_AC(hdf['Pf60']['variants']['AC'][1]))
print(num_AC(hdf['Pf60']['variants']['AC'][2]))
print(num_AC(hdf['Pf60']['variants']['AC'][7]))

1
2
2
3


In [74]:
num_alleles = [np.count_nonzero(values > 0) for values in hdf['Pf60']['variants']['AC']]

KeyboardInterrupt: 

In [77]:
%%time
def num_AC(values):
    np.count_nonzero(values > 0)

v_num_AC = np.vectorize(num_AC)

num_alleles = v_num_AC(hdf['Pf60']['variants']['AC'][:])

CPU times: user 52.2 s, sys: 700 ms, total: 52.9 s
Wall time: 52.9 s


In [79]:
num_alleles[0:10]

array([[None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None],
       [None, None, None, None, None, None]], dtype=object)

In [78]:
%%time
num_alleles2 = np.count_nonzero(hdf['Pf60']['variants']['AC'][:])

CPU times: user 540 ms, sys: 68 ms, total: 608 ms
Wall time: 604 ms


In [85]:
%%time
num_alleles3 = np.apply_along_axis(np.count_nonzero, 1, hdf['Pf60']['variants']['AC'][:])

CPU times: user 54.7 s, sys: 100 ms, total: 54.8 s
Wall time: 54.8 s


In [88]:
%%time
AC = np.sum(hdf['Pf60']['variants']['AC'][:], 1)

CPU times: user 624 ms, sys: 128 ms, total: 752 ms
Wall time: 750 ms


In [89]:
len(AC)

6051696

In [90]:
AC[0:10]

array([ 2, 14, 35, 31, 36, 43, 34, 37, 45, 37], dtype=uint64)

In [86]:
num_alleles3[0:10]

array([1, 2, 2, 2, 2, 2, 2, 3, 2, 3])

In [87]:
len(num_alleles3)

6051696

In [82]:
hdf['Pf60']['variants']['AC'][:]

array([[ 2,  0,  0,  0,  0,  0],
       [ 3, 11,  0,  0,  0,  0],
       [21, 14,  0,  0,  0,  0],
       ..., 
       [ 2,  2,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0],
       [ 1,  4,  0,  0,  0,  0]], dtype=uint16)

In [111]:
hdf['Pf60']['variants']['ALT'][:] != b'*'

array([[b'A', b'', b'', b'', b'', b''],
       [b'*', b'G', b'', b'', b'', b''],
       [b'A', b'*', b'', b'', b'', b''],
       ..., 
       [b'G', b'C', b'', b'', b'', b''],
       [b'G', b'', b'', b'', b'', b''],
       [b'T', b'G', b'', b'', b'', b'']], 
      dtype='|S600')

In [116]:
non_SD_AC = hdf['Pf60']['variants']['AC'][:]
non_SD_AC

array([[ 2,  0,  0,  0,  0,  0],
       [ 3, 11,  0,  0,  0,  0],
       [21, 14,  0,  0,  0,  0],
       ..., 
       [ 2,  2,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0],
       [ 1,  4,  0,  0,  0,  0]], dtype=uint16)

In [117]:
non_SD_AC[hdf['Pf60']['variants']['ALT'][:] == b'*'] = 0
non_SD_AC

array([[ 2,  0,  0,  0,  0,  0],
       [ 0, 11,  0,  0,  0,  0],
       [21,  0,  0,  0,  0,  0],
       ..., 
       [ 2,  2,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0],
       [ 1,  4,  0,  0,  0,  0]], dtype=uint16)

In [112]:
%%time
hdf['Pf60']['variants']['ALT'][:] != b'*'

CPU times: user 41.3 s, sys: 18.2 s, total: 59.5 s
Wall time: 59.5 s


array([[ True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True],
       ..., 
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]], dtype=bool)

In [81]:
num_alleles2

12170001

In [104]:
'FILTER_SubtelomericRepeat' in hdf['Pf60']['variants'].keys()

False

In [109]:
def num_unique(values):
    return(len(np.unique(values)))

def num_AC(values):
    np.count_nonzero(values > 0)

def determine_variant_counts_hdf(hdf5=hdf['Pf60'], VQSLOD_threshold=0.0, groupby_variables = ['VARIANT_TYPE', 'CDS'],
                                 ascending=None):
    number_of_samples = len(hdf5['samples'])
    
    if ascending is None:
        ascending = [True if x == 'MULTIALLELIC' else False for x in groupby_variables]
        
    df_variants = pd.DataFrame(
        {
            'CHROM': pd.Series(hdf5['variants']['CHROM'][:], dtype="category"),
            'POS': pd.Series(hdf5['variants']['POS'][:]),
            'num_alleles': pd.Series(np.apply_along_axis(np.count_nonzero, 1, hdf5['variants']['AC'][:])),
            'AC': pd.Series(np.sum(hdf5['variants']['AC'][:], 1)),
            'AN': pd.Series(hdf5['variants']['AN'][:]),
            'NRAF': pd.Series(np.sum(hdf5['variants']['AC'][:], 1) / hdf5['variants']['AN'][:]),
            'Missingness': pd.Series(((number_of_samples * 2) - hdf5['variants']['AN'][:]) / (number_of_samples * 2)),
            'CDS': pd.Series(hdf5['variants']['CDS'][:], dtype="category"),
            'VARIANT_TYPE': pd.Series(hdf5['variants']['VARIANT_TYPE'][:], dtype="category"),
            'MULTIALLELIC': pd.Series(hdf5['variants']['MULTIALLELIC'][:], dtype="category"),
        }
    )
#     For the following, Pf60 has FILTER_SubtelomericRepeat but Pv30 doesn't
    if 'FILTER_SubtelomericRepeat' in hdf5['variants'].keys():
        ix_good_calls = (
            (hdf5['variants']['VARIANT_TYPE'][:] != b'') &
            (~hdf5['variants']['FILTER_Centromere'][:]) &
            (~hdf5['variants']['FILTER_InternalHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericRepeat'][:]) &
            (hdf5['variants']['VQSLOD'][:] > VQSLOD_threshold)
        )
    else:
        ix_good_calls = (
            (hdf5['variants']['VARIANT_TYPE'][:] != b'') &
            (~hdf5['variants']['FILTER_Centromere'][:]) &
            (~hdf5['variants']['FILTER_InternalHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericHypervariable'][:]) &
            (hdf5['variants']['VQSLOD'][:] > VQSLOD_threshold)
        )

    df_variants['PASS'] = ix_good_calls
    
    for i, column in enumerate(['CHROM', 'VARIANT_TYPE', 'MULTIALLELIC']):
        df_variants[column].cat.categories = [
            x.decode('ascii') for x in df_variants[column].cat.categories
        ]
#     df_variants['CHROM_POS'] = df_variants['CHROM'].str.cat(df_variants['POS'].astype(str), sep='_')

    if len(groupby_variables) == 0:
#         temp = pd.DataFrame(variant_type_sum)
        df_variants['Total'] = 'Total'
        groupby_variables = ['Total']
    
#         temp.set_index('Total', append=True, inplace=True)
#         variant_type_sum = temp.ix[:, 0]
    
    gb_all = df_variants.groupby(groupby_variables)
    gb_pass = df_variants[df_variants['PASS']].groupby(groupby_variables)
    
    df_summary = pd.concat(
        [
            gb_all.size().rename("Discovered variant positions"),
            gb_pass.size().rename("Pass variant positions"),
            gb_pass['num_alleles'].sum().rename("Alleles"),
            gb_pass['NRAF'].mean().rename("Mean NRAF"),
            gb_pass['Missingness'].mean().rename("Missingness"),
        ],
        axis=1
    )
    df_summary["% pass"] = df_summary["Pass variant positions"] / df_summary["Discovered variant positions"]
    df_summary["Alleles per position"] = df_summary["Alleles"] / df_summary["Pass variant positions"]
    df_summary["Variants per sample"] = df_summary["Alleles"] * df_summary["Mean NRAF"]
    
    df_summary.sortlevel(range(len(groupby_variables)), ascending=ascending, inplace=True)
    df_summary.reset_index(inplace=True)
        
    if 'MULTIALLELIC' in df_summary.columns:
        df_summary = df_summary[df_summary['MULTIALLELIC'] != 'MIS']
    
    return(df_summary)
    


In [123]:
def num_unique(values):
    return(len(np.unique(values)))

def num_AC(values):
    np.count_nonzero(values > 0)

def determine_variant_counts_hdf(hdf5=hdf['Pf60'], VQSLOD_threshold=0.0, groupby_variables = ['VARIANT_TYPE', 'CDS'],
                                 ascending=None):
    number_of_samples = len(hdf5['samples'])
    
    if ascending is None:
        ascending = [True if x == 'MULTIALLELIC' else False for x in groupby_variables]
        
    non_SD_AC = hdf5['variants']['AC'][:]
    non_SD_AC[hdf5['variants']['ALT'][:] == b'*'] = 0
    
    df_variants = pd.DataFrame(
        {
            'CHROM': pd.Series(hdf5['variants']['CHROM'][:], dtype="category"),
            'POS': pd.Series(hdf5['variants']['POS'][:]),
            'num_alleles': pd.Series(np.apply_along_axis(np.count_nonzero, 1, non_SD_AC)),
            'AC': pd.Series(np.sum(non_SD_AC, 1)),
            'AN': pd.Series(hdf5['variants']['AN'][:] * pd.Series(np.apply_along_axis(np.count_nonzero, 1, non_SD_AC))),
#             'NRAF': pd.Series(np.sum(non_SD_AC, 1) / hdf5['variants']['AN'][:]),
            'Missingness': pd.Series(((number_of_samples * 2) - hdf5['variants']['AN'][:]) / (number_of_samples * 2)),
            'CDS': pd.Series(hdf5['variants']['CDS'][:], dtype="category"),
            'VARIANT_TYPE': pd.Series(hdf5['variants']['VARIANT_TYPE'][:], dtype="category"),
            'MULTIALLELIC': pd.Series(hdf5['variants']['MULTIALLELIC'][:], dtype="category"),
        }
    )
#     For the following, Pf60 has FILTER_SubtelomericRepeat but Pv30 doesn't
    if 'FILTER_SubtelomericRepeat' in hdf5['variants'].keys():
        ix_good_calls = (
            (hdf5['variants']['VARIANT_TYPE'][:] != b'') &
            (~hdf5['variants']['FILTER_Centromere'][:]) &
            (~hdf5['variants']['FILTER_InternalHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericRepeat'][:]) &
            (hdf5['variants']['VQSLOD'][:] > VQSLOD_threshold)
        )
    else:
        ix_good_calls = (
            (hdf5['variants']['VARIANT_TYPE'][:] != b'') &
            (~hdf5['variants']['FILTER_Centromere'][:]) &
            (~hdf5['variants']['FILTER_InternalHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericHypervariable'][:]) &
            (hdf5['variants']['VQSLOD'][:] > VQSLOD_threshold)
        )

    df_variants['PASS'] = ix_good_calls
    
    for i, column in enumerate(['CHROM', 'VARIANT_TYPE', 'MULTIALLELIC']):
        df_variants[column].cat.categories = [
            x.decode('ascii') for x in df_variants[column].cat.categories
        ]
#     df_variants['CHROM_POS'] = df_variants['CHROM'].str.cat(df_variants['POS'].astype(str), sep='_')

    if len(groupby_variables) == 0:
#         temp = pd.DataFrame(variant_type_sum)
        df_variants['Total'] = 'Total'
        groupby_variables = ['Total']
    
#         temp.set_index('Total', append=True, inplace=True)
#         variant_type_sum = temp.ix[:, 0]
    
    gb_all = df_variants.groupby(groupby_variables)
    gb_pass = df_variants[df_variants['PASS']].groupby(groupby_variables)
    
    df_summary = pd.concat(
        [
            gb_all.size().rename("Discovered variant positions"),
            gb_pass.size().rename("Pass variant positions"),
            gb_pass['num_alleles'].sum().rename("Alleles"),
            gb_pass['AC'].sum().rename("Total AC"),
            gb_pass['AN'].sum().rename("Total AN"),
#             gb_pass['NRAF'].mean().rename("Mean NRAF"),
            gb_pass['Missingness'].mean().rename("Missingness"),
        ],
        axis=1
    )
    df_summary["Mean NRAF"] = df_summary["Total AC"] / df_summary["Total AN"]    
    df_summary["% pass"] = df_summary["Pass variant positions"] / df_summary["Discovered variant positions"]
    df_summary["Alleles per position"] = df_summary["Alleles"] / df_summary["Pass variant positions"]
    df_summary["Variants per sample"] = df_summary["Alleles"] * df_summary["Mean NRAF"]
    
    df_summary.sortlevel(range(len(groupby_variables)), ascending=ascending, inplace=True)
    df_summary.reset_index(inplace=True)
        
    if 'MULTIALLELIC' in df_summary.columns:
        df_summary = df_summary[df_summary['MULTIALLELIC'] != 'MIS']
    
    return(df_summary)
    


In [123]:
def num_unique(values):
    return(len(np.unique(values)))

def num_AC(values):
    np.count_nonzero(values > 0)

def determine_variant_counts_hdf(hdf5=hdf['Pf60'], VQSLOD_threshold=0.0, groupby_variables = ['VARIANT_TYPE', 'CDS'],
                                 ascending=None):
    number_of_samples = len(hdf5['samples'])
    
    if ascending is None:
        ascending = [True if x == 'MULTIALLELIC' else False for x in groupby_variables]
        
    non_SD_AC = hdf5['variants']['AC'][:]
    non_SD_AC[hdf5['variants']['ALT'][:] == b'*'] = 0
    
    df_variants = pd.DataFrame(
        {
#             'CHROM': pd.Series(hdf5['variants']['CHROM'][:], dtype="category"),
#             'POS': pd.Series(hdf5['variants']['POS'][:]),
            'num_alleles': pd.Series(np.apply_along_axis(np.count_nonzero, 1, non_SD_AC)),
            'AC': pd.Series(np.sum(non_SD_AC, 1)),
            'AN': pd.Series(hdf5['variants']['AN'][:] * pd.Series(np.apply_along_axis(np.count_nonzero, 1, non_SD_AC))),
            'Missingness': pd.Series(((number_of_samples * 2) - hdf5['variants']['AN'][:]) / (number_of_samples * 2)),
            'CDS': pd.Series(hdf5['variants']['CDS'][:], dtype="category"),
            'VARIANT_TYPE': pd.Series(hdf5['variants']['VARIANT_TYPE'][:], dtype="category"),
            'MULTIALLELIC': pd.Series(hdf5['variants']['MULTIALLELIC'][:], dtype="category"),
        }
    )
#     For the following, Pf60 has FILTER_SubtelomericRepeat but Pv30 doesn't
    if 'FILTER_SubtelomericRepeat' in hdf5['variants'].keys():
        ix_good_calls = (
            (hdf5['variants']['VARIANT_TYPE'][:] != b'') &
            (~hdf5['variants']['FILTER_Centromere'][:]) &
            (~hdf5['variants']['FILTER_InternalHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericRepeat'][:]) &
            (hdf5['variants']['VQSLOD'][:] > VQSLOD_threshold)
        )
    else:
        ix_good_calls = (
            (hdf5['variants']['VARIANT_TYPE'][:] != b'') &
            (~hdf5['variants']['FILTER_Centromere'][:]) &
            (~hdf5['variants']['FILTER_InternalHypervariable'][:]) &
            (~hdf5['variants']['FILTER_SubtelomericHypervariable'][:]) &
            (hdf5['variants']['VQSLOD'][:] > VQSLOD_threshold)
        )

    df_variants['PASS'] = ix_good_calls
    
    for i, column in enumerate(['VARIANT_TYPE', 'MULTIALLELIC']):
        df_variants[column].cat.categories = [
            x.decode('ascii') for x in df_variants[column].cat.categories
        ]

    if len(groupby_variables) == 0:
        df_variants['Total'] = 'Total'
        groupby_variables = ['Total']
    
    
    gb_all = df_variants.groupby(groupby_variables)
    gb_pass = df_variants[df_variants['PASS']].groupby(groupby_variables)
    
    df_summary = pd.concat(
        [
            gb_all.size().rename("Discovered variant positions"),
            gb_pass.size().rename("Pass variant positions"),
            gb_pass['num_alleles'].sum().rename("Alleles"),
            gb_pass['AC'].sum().rename("Total AC"),
            gb_pass['AN'].sum().rename("Total AN"),
            gb_pass['Missingness'].mean().rename("Missingness"),
        ],
        axis=1
    )
    df_summary["Mean NRAF"] = df_summary["Total AC"] / df_summary["Total AN"]    
    df_summary["% pass"] = df_summary["Pass variant positions"] / df_summary["Discovered variant positions"]
    df_summary["Alleles per position"] = df_summary["Alleles"] / df_summary["Pass variant positions"]
    df_summary["Variants per sample"] = df_summary["Alleles"] * df_summary["Mean NRAF"]
    
    df_summary.sortlevel(range(len(groupby_variables)), ascending=ascending, inplace=True)
    df_summary.reset_index(inplace=True)
        
    if 'MULTIALLELIC' in df_summary.columns:
        df_summary = df_summary[df_summary['MULTIALLELIC'] != 'MIS']
    
    return(df_summary)
    


# Pf 6.0 variant counts

In [110]:
%%time
# This version was run before requiring ALT!='*'
determine_variant_counts_hdf()

CPU times: user 1min 2s, sys: 2.46 s, total: 1min 5s
Wall time: 1min 5s


,VARIANT_TYPE,CDS,Discovered variant positions,Pass variant positions,Alleles,Mean NRAF,Missingness,% pass,Alleles per position,Variants per sample
0,SNP,True,1996153,1319276,1618268,0.007966,0.034135,0.660909,1.226633,12891.904221
1,SNP,False,1850432,858034,1151368,0.028157,0.086311,0.463694,1.341868,32419.209077
2,INDEL,True,672155,192190,576676,0.083389,0.060883,0.285931,3.000552,48088.650097
3,INDEL,False,1532956,745260,1971048,0.096900,0.172933,0.486159,2.644779,190994.502160


In [119]:
%%time
# This was when Mean NRAF by position, rather than by allele
determine_variant_counts_hdf()

CPU times: user 1min 50s, sys: 19.8 s, total: 2min 10s
Wall time: 2min 9s


,VARIANT_TYPE,CDS,Discovered variant positions,Pass variant positions,Alleles,Mean NRAF,Missingness,% pass,Alleles per position,Variants per sample
0,SNP,True,1996153,1319276,1483638,0.004887,0.034135,0.660909,1.124585,7250.456961
1,SNP,False,1850432,858034,947421,0.006087,0.086311,0.463694,1.104177,5766.878858
2,INDEL,True,672155,192190,428081,0.029783,0.060883,0.285931,2.227384,12749.343709
3,INDEL,False,1532956,745260,1507015,0.035887,0.172933,0.486159,2.022133,54082.231567


In [124]:
%%time
determine_variant_counts_hdf()

CPU times: user 2min 47s, sys: 20.2 s, total: 3min 7s
Wall time: 3min 7s


,VARIANT_TYPE,CDS,Discovered variant positions,Pass variant positions,Alleles,Total AC,Total AN,Missingness,Mean NRAF,% pass,Alleles per position,Variants per sample
0,SNP,True,1996153,1319276,1483638,87186083,20582004452,0.034135,0.004236,0.660909,1.124585,6284.741902
1,SNP,False,1850432,858034,947421,64233184,12452397460,0.086311,0.005158,0.463694,1.104177,4887.080389
2,INDEL,True,672155,192190,428081,74461841,5757249718,0.060883,0.012934,0.285931,2.227384,5536.619205
3,INDEL,False,1532956,745260,1507015,306226303,17778067670,0.172933,0.017225,0.486159,2.022133,25958.256014


In [102]:
%%time
determine_variant_counts()

CPU times: user 1min 38s, sys: 4.84 s, total: 1min 43s
Wall time: 1min 43s


,VARIANT_TYPE,CDS,Discovered variant positions,Pass variant positions,Alleles,Mean NRAF,Missingness,% pass,Alleles per position,Variants per sample
0,SNP,True,1996153,1319276.0,1483638.0,0.004346,0.034206,0.660909,1.124585,6447.228945
1,SNP,False,1850432,858034.0,947421.0,0.005513,0.084972,0.463694,1.104177,5222.787054
2,IND,True,672155,192190.0,428081.0,0.013371,0.063703,0.285931,2.227384,5723.908250
3,IND,False,1532955,745260.0,1507014.0,0.017747,0.178719,0.486159,2.022132,26745.137765
4,,False,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
determine_variant_counts(groupby_variables=[])

,Total,Discovered variant positions,Pass variant positions,Alleles,Mean NRAF,Missingness,% pass,Alleles per position,Variants per sample
0,Total,6051698,3114760,4366154,0.010109,0.097994,0.514692,1.401763,44139.062014


In [125]:
determine_variant_counts_hdf(groupby_variables=[])

,Total,Discovered variant positions,Pass variant positions,Alleles,Total AC,Total AN,Missingness,Mean NRAF,% pass,Alleles per position,Variants per sample
0,Total,6051696,3114760,4366155,532107411,56569719300,0.083369,0.009406,0.514692,1.401763,41069.028834


In [132]:
%%time
determine_variant_counts(groupby_variables = ['VARIANT_TYPE', 'CDS', 'MULTIALLELIC'])

,VARIANT_TYPE,CDS,MULTIALLELIC,Discovered variant positions,Pass variant positions,Alleles,Mean NRAF,Missingness,% pass,Alleles per position,Variants per sample
0,SNP,True,BI,1590717,1044869.0,1044869.0,0.004359,0.032077,0.656854,1.000000,4554.065283
1,SNP,True,MU,236939,155084.0,319446.0,0.004191,0.034711,0.654531,2.059826,1338.787576
2,SNP,True,SD,168497,119323.0,119323.0,0.004646,0.051499,0.708161,1.000000,554.376087
3,SNP,False,BI,1203255,582017.0,582017.0,0.005051,0.065618,0.483702,1.000000,2939.619675
4,SNP,False,MU,266915,85097.0,174484.0,0.005298,0.072221,0.318817,2.050413,924.360164
5,SNP,False,SD,380262,190920.0,190920.0,0.007117,0.155623,0.502075,1.000000,1358.807215
6,IND,True,BI,80037,19059.0,19059.0,0.004266,0.041287,0.238127,1.000000,81.309240
7,IND,True,MU,480615,109642.0,345533.0,0.015204,0.065170,0.228129,3.151466,5253.590294
8,IND,True,SD,111503,63489.0,63489.0,0.006127,0.062447,0.569393,1.000000,389.008716
9,IND,False,BI,254006,139792.0,139792.0,0.003324,0.097701,0.550349,1.000000,464.615163


In [133]:
%%time
determine_variant_counts_hdf(groupby_variables = ['VARIANT_TYPE', 'CDS', 'MULTIALLELIC'])

CPU times: user 2min 47s, sys: 21 s, total: 3min 8s
Wall time: 3min 9s


,VARIANT_TYPE,CDS,MULTIALLELIC,Discovered variant positions,Pass variant positions,Alleles,Total AC,Total AN,Missingness,Mean NRAF,% pass,Alleles per position,Variants per sample
0,SNP,True,BI,1590717,1044869,1044869,62283667,14527066052,0.032077,0.004287,0.656854,1.000000,4479.794655
1,SNP,True,MU,236939,155084,319446,18033715,4429250150,0.034641,0.004072,0.654531,2.059826,1300.626049
2,SNP,True,SD,168497,119323,119323,6868701,1625688250,0.051499,0.004225,0.708161,1.000000,504.152016
3,SNP,False,BI,1203255,582017,582017,37969142,7811517188,0.065618,0.004861,0.483702,1.000000,2828.987710
4,SNP,False,MU,266915,85097,174484,11463610,2325281270,0.072337,0.004930,0.318817,2.050413,860.204119
5,SNP,False,SD,380262,190920,190920,14800432,2315599002,0.155623,0.006392,0.502075,1.000000,1220.288347
6,INDEL,True,BI,80037,19059,19059,1093481,262460724,0.041287,0.004166,0.238127,1.000000,79.404850
7,INDEL,True,MU,480615,109642,345533,68364740,4639781854,0.063384,0.014734,0.228129,3.151466,5091.246625
8,INDEL,True,SD,111503,63489,63489,5003620,855007140,0.062447,0.005852,0.569393,1.000000,371.546406
9,INDEL,False,BI,254007,139792,139792,5726895,1811791500,0.097701,0.003161,0.550347,1.000000,441.868783


In [137]:
%%time
group_by_options = collections.OrderedDict()
group_by_options['Total'] = []
group_by_options['Summary'] = ['VARIANT_TYPE', 'CDS']
group_by_options['Multiallelic'] = ['VARIANT_TYPE', 'CDS', 'MULTIALLELIC']

for release in hdf:
    writer = pd.ExcelWriter('%s/variant_summaries_%s.xlsx' % (output_dir, release))
    for group_by_option_name in group_by_options:
        print(release, group_by_option_name)
        (
            determine_variant_counts_hdf(hdf[release], groupby_variables=group_by_options[group_by_option_name])
            .to_excel(writer, group_by_option_name)
        )
    writer.save()

Pf60 Total
Pf60 Summary
Pf60 Multiallelic
Pv30 Total
Pv30 Summary
Pv30 Multiallelic
CPU times: user 14min 11s, sys: 1min 41s, total: 15min 52s
Wall time: 15min 53s


In [4]:
!cp {output_dir}/variant_summaries_Pf60.xlsx /nfs/team112_internal/production/release_build/Pf/6_0_release_packages/variant_summaries_pf_60.xlsx

In [5]:
!cp {output_dir}/variant_summaries_Pv30.xlsx /nfs/team112_internal/production/release_build/Pv/3_0_release_packages/variant_summaries_pv_30.xlsx

# Sanity check

In [6]:
pf60_biallelic_pass_coding_snps = (
    (hdf['Pf60']['variants']['FILTER_PASS'][:]) &
    (hdf['Pf60']['variants']['VARIANT_TYPE'][:] == b'SNP') &
    (hdf['Pf60']['variants']['CDS'][:]) &
    (hdf['Pf60']['variants']['MULTIALLELIC'][:] == b'BI')
)
np.count_nonzero(pf60_biallelic_pass_coding_snps)

1044869

In [16]:
np.mean(hdf['Pf60']['variants']['AF'][:][pf60_biallelic_pass_coding_snps])

0.00072289

In [8]:
pf60_biallelic_pass_VQSLOD6_coding_snps = (
    (hdf['Pf60']['variants']['FILTER_PASS'][:]) &
    (hdf['Pf60']['variants']['VARIANT_TYPE'][:] == b'SNP') &
    (hdf['Pf60']['variants']['CDS'][:]) &
    (hdf['Pf60']['variants']['MULTIALLELIC'][:] == b'BI') &
    (hdf['Pf60']['variants']['VQSLOD'][:] >= 6.0)
)
np.count_nonzero(pf60_biallelic_pass_VQSLOD6_coding_snps)

83195

In [17]:
np.mean(hdf['Pf60']['variants']['AF'][:][pf60_biallelic_pass_VQSLOD6_coding_snps])

0.0070686

In [10]:
pf60_biallelic_pass_VQSLOD6_coding_indels = (
    (hdf['Pf60']['variants']['FILTER_PASS'][:]) &
    (hdf['Pf60']['variants']['VARIANT_TYPE'][:] == b'INDEL') &
    (hdf['Pf60']['variants']['CDS'][:]) &
    (hdf['Pf60']['variants']['MULTIALLELIC'][:] == b'BI') &
    (hdf['Pf60']['variants']['VQSLOD'][:] >= 6.0)
)
np.count_nonzero(pf60_biallelic_pass_VQSLOD6_coding_indels)

39

In [12]:
pv30_biallelic_pass_coding_snps = (
    (hdf['Pv30']['variants']['FILTER_PASS'][:]) &
    (hdf['Pv30']['variants']['VARIANT_TYPE'][:] == b'SNP') &
    (hdf['Pv30']['variants']['CDS'][:]) &
    (hdf['Pv30']['variants']['MULTIALLELIC'][:] == b'BI')
)
np.count_nonzero(pv30_biallelic_pass_coding_snps)

407611

In [14]:
pv30_biallelic_pass_VQSLOD6_coding_snps = (
    (hdf['Pv30']['variants']['FILTER_PASS'][:]) &
    (hdf['Pv30']['variants']['VARIANT_TYPE'][:] == b'SNP') &
    (hdf['Pv30']['variants']['CDS'][:]) &
    (hdf['Pv30']['variants']['MULTIALLELIC'][:] == b'BI') &
    (hdf['Pv30']['variants']['VQSLOD'][:] >= 6.0)
)
np.count_nonzero(pv30_biallelic_pass_VQSLOD6_coding_snps)

126011

In [15]:
pv30_biallelic_pass_VQSLOD6_coding_indels = (
    (hdf['Pv30']['variants']['FILTER_PASS'][:]) &
    (hdf['Pv30']['variants']['VARIANT_TYPE'][:] == b'INDEL') &
    (hdf['Pv30']['variants']['CDS'][:]) &
    (hdf['Pv30']['variants']['MULTIALLELIC'][:] == b'BI') &
    (hdf['Pv30']['variants']['VQSLOD'][:] >= 6.0)
)
np.count_nonzero(pv30_biallelic_pass_VQSLOD6_coding_indels)

2406

In [4]:
list(hdf.keys())

['Pf60', 'Pv30']

# Junk
The following were some attempts to find out why the numbers didn't quite match the final VCFs (and in particular, why a get variants didn't get annotated as SNP or INDEL. I didn't fully get to the bottom of this, but it seems like some new positions are created when using bcftools norm. I decided the results based on the position-level HDF5, rather than the allele-level VCF were more reliable.

In [128]:
ix_good_calls = (
    (hdf['Pf60']['variants']['VARIANT_TYPE'][:] != b'') &
    (~hdf['Pf60']['variants']['FILTER_Centromere'][:]) &
    (~hdf['Pf60']['variants']['FILTER_InternalHypervariable'][:]) &
    (~hdf['Pf60']['variants']['FILTER_SubtelomericHypervariable'][:]) &
    (~hdf['Pf60']['variants']['FILTER_SubtelomericRepeat'][:]) &
    (hdf['Pf60']['variants']['VQSLOD'][:] > 0.0)
)

In [129]:
pd.value_counts(ix_good_calls)

True     3114760
False    2936936
dtype: int64

In [15]:
1319276.0 + 858034.0 + 192190.0 + 745260.0

3114760.0

In [34]:
1996153 + 1850432 + 672155 + 1532955

6051695

In [35]:
1996153 + 1850432 + 672155 + 1532955 + 35

6051730

In [49]:
!zgrep -v '^#' /nfs/team112_internal/production_files/Pf/6_0/SNP_INDEL_*.combined.filtered.vcf.gz | wc -l 

6051696


In [130]:
!zgrep -v '^#' /nfs/team112_internal/production_files/Pf/6_0/SNP_INDEL_*.combined.filtered.vcf.gz | grep -P '\tPASS\t' | wc -l 

3114760


In [50]:
# This is wrong!
!zgrep -v -P '\tPASS\t' /nfs/team112_internal/production_files/Pf/6_0/SNP_INDEL_*.combined.filtered.vcf.gz | wc -l 

2939704


In [54]:
hdf['Pf60']['variants']['CHROM']

<HDF5 dataset "CHROM": shape (6051696,), type "|S12">

# Why do some variants have no VARIANT_TYPE?

In [36]:
pd.value_counts(variants['Pf60']['VARIANT_TYPE'])

b'IND'    5537629
b'SNP'    4425589
b''            35
dtype: int64

In [40]:
np.array([len(seq) for seq in variants['Pf60']['REF'][variants['Pf60']['VARIANT_TYPE'] == b'']])

array([347, 304, 289, 232, 359, 271, 336, 376, 304, 335, 265, 307, 308,
       293, 275, 291, 290, 354, 271, 316, 295, 283, 269, 295, 295, 274,
       339, 341, 332, 333, 344, 292, 314, 297, 290])

In [41]:
np.array([len(seq) for seq in variants['Pf60']['ALT'][variants['Pf60']['VARIANT_TYPE'] == b'']])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [45]:
list(zip(
    variants['Pf60']['CHROM'][variants['Pf60']['VARIANT_TYPE'] == b''],
    variants['Pf60']['POS'][variants['Pf60']['VARIANT_TYPE'] == b'']
))

[(b'Pf3D7_01_v3', 7113),
 (b'Pf3D7_01_v3', 26128),
 (b'Pf3D7_01_v3', 26437),
 (b'Pf3D7_01_v3', 637475),
 (b'Pf3D7_02_v3', 16095),
 (b'Pf3D7_02_v3', 944225),
 (b'Pf3D7_03_v3', 15569),
 (b'Pf3D7_03_v3', 21953),
 (b'Pf3D7_03_v3', 1051532),
 (b'Pf3D7_03_v3', 1067309),
 (b'Pf3D7_04_v3', 609963),
 (b'Pf3D7_06_v3', 60280),
 (b'Pf3D7_06_v3', 1307742),
 (b'Pf3D7_07_v3', 16473),
 (b'Pf3D7_07_v3', 1080971),
 (b'Pf3D7_08_v3', 375),
 (b'Pf3D7_08_v3', 1456356),
 (b'Pf3D7_09_v3', 1516630),
 (b'Pf3D7_10_v3', 3203),
 (b'Pf3D7_10_v3', 13442),
 (b'Pf3D7_10_v3', 13474),
 (b'Pf3D7_10_v3', 1536049),
 (b'Pf3D7_10_v3', 1536063),
 (b'Pf3D7_10_v3', 1664694),
 (b'Pf3D7_10_v3', 1664701),
 (b'Pf3D7_10_v3', 1664715),
 (b'Pf3D7_10_v3', 1674229),
 (b'Pf3D7_10_v3', 1674235),
 (b'Pf3D7_10_v3', 1674239),
 (b'Pf3D7_12_v3', 10650),
 (b'Pf3D7_12_v3', 2261954),
 (b'Pf3D7_13_v3', 2902737),
 (b'Pf3D7_13_v3', 2904906),
 (b'Pf3D7_13_v3', 2911564),
 (b'Pf3D7_13_v3', 2911574)]

In [48]:
variants['Pf60'][(variants['Pf60']['CHROM'] == b'Pf3D7_01_v3') & (variants['Pf60']['POS'] == 7113)]

array([ (b'Pf3D7_01_v3', 7113, b'G', b'A', False, b'SNP', b'SD', 59, 11586, False, False, False, False, False, False, True, False, False, False, False, False, 2.5999999046325684),
       (b'Pf3D7_01_v3', 7113, b'gACCGTTCTATTAGTCAAAAAACGACATCCAATTTATAGGGGAAACAAAACCGGAAATTCGGGTTTTAGACCCAAATTATAGAACGCAATTTCCACCAGGTACGCCAACGTGGATATTTCTATCATATAGTCCGTGATCAACTATACATACTACGTGGTACCACATGGTTACTACATCACACGCGTGGTGTTTAATCACCATTTTTTCATTATACCCAATTTTCCCCTAGCACTTTTGGTGCACAATTCACATGCACTTGAAAGCATTTGATGATTATGATGTTGCACACACGATGTTAACATCTCAATCAAAATTGTGGGTCACTTACTATATGGTAATATACCCT', b'g', False, b'', b'', 1, 12222, False, False, False, False, False, False, True, False, False, False, False, False, 2.759999990463257)], 
      dtype=[('CHROM', 'S12'), ('POS', '<i4'), ('REF', 'S600'), ('ALT', 'S600'), ('CDS', '?'), ('VARIANT_TYPE', 'S3'), ('MULTIALLELIC', 'S2'), ('AC', '<u2'), ('AN', '<u2'), ('FILTER_Centromere', '?'), ('FILTER_InternalHypervariable', '?'), ('FILTER_LowQual', '?'), ('FILTER_Low_VQSLOD', '?'), ('FILTE

In [47]:
!tabix /nfs/team112_internal/production_files/Pf/6_0/SNP_INDEL_Pf3D7_01_v3.combined.filtered.vcf.gz Pf3D7_01_v3:7103-7123

Pf3D7_01_v3	6987	.	GCCGTTTTTGGGCAGGGAAAAGGTGAGTCAATTGTTCCTTTACATATACATTTATCACTGTTGTGGTAATAGATATATATAGCTTATGGTCCAGAATTTATATAGTGACTCTTTGTTGTCCCTTGGGACCGTTCTATTAGTCAAAAAACGACATCCAATTTATAGGGGAAACAAAACCGGAAATTCGGGTTTTAGA	CCCGTTTTTGGGCAGGGAAAAGGTGAGTCAATTGTTCCTTTACATATACATTTATCACTGTTGTGGTAATAGATATATATAGCTTATGGTCCAGAATTTATATAGTGACTCTTTGTTGTCCCTTGGGACCGTTCTATTAGTCAAAAAACGACATCCAATTTATAGGGGAAACAAAACCGGAAATTCGGGTTTTAGA,*,TCCGTTTTTGGGCAGGGAAAAGGTGAGTCAATTGTTCCTTTACATATACATTTATCACTGTTGTGGTAATAGATATATATAGCTTATGGTCCAGAATTTATATAGTGACTCTTTGTTGTCCCTTGGGACCGTTCTATTAGTCAAAAAACGACATCCAATTTATAGGGGAAACAAAACCGGAAATTCGGGTTTTAGA,ACCGTTTTTGGGCAGGGAAAAGGTGAGTCAATTGTTCCTTTACATATACATTTATCACTGTTGTGGTAATAGATATATATAGCTTATGGTCCAGAATTTATATAGTGACTCTTTGTTGTCCCTTGGGACCGTTCTATTAGTCAAAAAACGACATCCAATTTATAGGGGAAACAAAACCGGAAATTCGGGTTTTAGA,GCGTTTTTGGGCAGGGAAAAGGTGAGTCAATTGTTCCTTTACATATACATTTATCACTGTTGTGGTAATAGATATATATAGCTTATGGTCCAGAATTTATATAGTGACTCTTTGTTGTCCCTTGGGACCGTTCTATTAGTCAAAAAACGACATCCAATTTATAGGGGAAACAAAACCGGAAATTCGGGTTT

In [46]:
!tabix /lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161125_Pf60_final_vcfs/intermediate_files/\
SNP_INDEL_Pf3D7_01_v3.combined.filtered.indels.txt.gz Pf3D7_01_v3:7103-7123

Pf3D7_01_v3	7103	TG	GG,*,TCGG,AG,CG	INDEL
Pf3D7_01_v3	7104	G	A,GCGC,GC,C,GCGA	INDEL
Pf3D7_01_v3	7109	TTGGG	T	INDEL
Pf3D7_01_v3	7110	TGGGACCG	GGGGACCG,*,CGGGACCG	INDEL
Pf3D7_01_v3	7111	GGGACCGTTCTATTAGTC	*,AGGACCGTTCTATTAGTC,CGGACCGTTCTATTAGTC	INDEL
Pf3D7_01_v3	7114	ACCGTTCTATTAGTCAAAAAACGACATCCAATTTATAGGGGAAACAAAACCGGAAATTCGGGTTTTAGACCCAAATTATAGAACGCAATTTCCACCAGGTACGCCAACGTGGATATTTCTATCATATAGTCCGTGATCAACTATACATACTACGTGGTACCACATGGTTACTACATCACACGCGTGGTGTTTAATCACCATTTTTTCATTATACCCAATTTTCCCCTAGCACTTTTGGTGCACAATTCACATGCACTTGAAAGCATTTGATGATTATGATGTTGCACACACGATGTTAACATCTCAATCAAAATTGTGGGTCACTTACTATATGGTAATATACCCTA	A	INDEL
Pf3D7_01_v3	7120	CTATTAGTCAAAAAACGA	TTATTAGTCAAAAAACGA,ATATTAGTCAAAAAACGA,GTATTAGTCAAAAAACGA,*	INDEL
Pf3D7_01_v3	7121	TATTA	CATTA,*	INDEL


In [42]:
variants['Pf60'].dtype.names

('CHROM',
 'POS',
 'REF',
 'ALT',
 'CDS',
 'VARIANT_TYPE',
 'MULTIALLELIC',
 'AC',
 'AN',
 'FILTER_Centromere',
 'FILTER_InternalHypervariable',
 'FILTER_LowQual',
 'FILTER_Low_VQSLOD',
 'FILTER_PASS',
 'FILTER_SubtelomericHypervariable',
 'FILTER_SubtelomericRepeat',
 'FILTER_VQSRTrancheINDEL99.00to99.90',
 'FILTER_VQSRTrancheINDEL99.90to100.00',
 'FILTER_VQSRTrancheINDEL99.90to100.00+',
 'FILTER_VQSRTrancheSNP99.90to100.00',
 'FILTER_VQSRTrancheSNP99.90to100.00+',
 'VQSLOD')

In [9]:
determine_variant_counts(groupby_variables = ['VARIANT_TYPE', 'CDS', 'MULTIALLELIC'])

,VARIANT_TYPE,CDS,MULTIALLELIC,Discovered variant positions,Pass variant positions,Alleles,Mean NRAF,Missingness,% pass,Alleles per position,Variants per sample
0,SNP,True,BI,1590717,1044869.0,1044869.0,0.004359,0.032077,0.656854,1.000000,4554.065283
1,SNP,True,MU,236939,155084.0,319446.0,0.004191,0.034711,0.654531,2.059826,1338.787576
2,SNP,True,SD,168497,119323.0,119323.0,0.004646,0.051499,0.708161,1.000000,554.376087
3,SNP,False,BI,1203255,582017.0,582017.0,0.005051,0.065618,0.483702,1.000000,2939.619675
4,SNP,False,MU,266915,85097.0,174484.0,0.005298,0.072221,0.318817,2.050413,924.360164
5,SNP,False,SD,380262,190920.0,190920.0,0.007117,0.155623,0.502075,1.000000,1358.807215
6,IND,True,BI,80037,19059.0,19059.0,0.004266,0.041287,0.238127,1.000000,81.309240
7,IND,True,MU,480615,109642.0,345533.0,0.015204,0.065170,0.228129,3.151466,5253.590294
8,IND,True,SD,111503,63489.0,63489.0,0.006127,0.062447,0.569393,1.000000,389.008716
9,IND,False,BI,254006,139792.0,139792.0,0.003324,0.097701,0.550349,1.000000,464.615163


# Pv 3.0 variant counts

In [22]:
determine_variant_counts()

,VARIANT_TYPE,CDS,Discovered variant positions,Pass variant positions,Alleles,Mean NRAF,Missingness,% pass,Alleles per position,Variants per sample
0,SNP,True,1996153,1319276.0,1483638.0,0.004346,0.034206,0.660909,1.124585,6447.228945
1,SNP,False,1850432,858034.0,947421.0,0.005513,0.084972,0.463694,1.104177,5222.787054
2,IND,True,672155,192190.0,428081.0,0.013371,0.063703,0.285931,2.227384,5723.908250
3,IND,False,1532955,745260.0,1507014.0,0.017747,0.178719,0.486159,2.022132,26745.137765
4,,False,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
determine_variant_counts(variants=variants['Pv30'], number_of_samples=number_of_samples['Pv30'])

,VARIANT_TYPE,CDS,Discovered variant positions,Pass variant positions,Alleles,Mean NRAF,Missingness,% pass,Alleles per position,Variants per sample
0,SNP,True,965032,509651.0,551406.0,0.037570,0.116504,0.528118,1.081929,20716.409945
1,SNP,False,1349958,629874.0,777831.0,0.053374,0.167917,0.466588,1.234899,41515.749754
2,IND,True,248520,106399.0,215480.0,0.047035,0.230220,0.428131,2.025207,10135.156734
3,IND,False,499616,197501.0,397067.0,0.054896,0.244291,0.395306,2.010456,21797.279569
4,,False,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
determine_variant_counts(variants=variants['Pv30'], number_of_samples=number_of_samples['Pv30'])

In [ ]:
determine_variant_counts(variants=variants['Pv30'], number_of_samples=number_of_samples['Pv30'])

In [30]:
'FILTER_SubtelomericRepeat' in variants['Pf60'].dtype.names

True

In [33]:
105800-63068

42732

In [10]:
# !vcf2npy \
#     --vcf {normalised_annotated_wg_vcf_fn} \
#     --fasta {genome_fn} \
#     --output-dir {'%s.vcfnp_cache' % normalised_annotated_wg_vcf_fn} \
#     --array-type variants \
#     --progress 100000 \
#     --arity ALT:1 \
#     --arity AC:1 \
#     --arity svlen:1 \
#     --dtype REF:a300 \
#     --dtype ALT:a300 \
#     --dtype MULTIALLELIC:a2 \
#     --dtype VARIANT_TYPE:a5 \
#     --exclude-field AF